# Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this notebook we will explore and cluster the Neighborhoods in Toronto.
Following will be implemented:
- Dataframe with Postal Code, Borough and Neighborhood
    * The Datafame has below features:
        1. Cells with a borough that is Not assigned are being ignored.
        2. Comma seperated Neighbourhoods for same Postal Code
        3. For a with Not assigned neighborhood, the neighborhood is updated to be same as the borough
- Latitude and Longitude coordinates (using Geocoder Python package) have been added for each Neighborhood
- Explore and cluster the neighborhoods in Toronto

In [191]:
import pandas as pd
import lxml.html as lh
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [192]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [193]:
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [194]:
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[:]:
    i+=1
    name=t.text_content()
    tup=name.split('\n')
    col.append((tup))

In [195]:
df = pd.DataFrame(col)

df.drop(df.iloc[:, 4:66], inplace = True, axis = 1) 
df.iloc[:,0]='ToBeDeleted'
df.drop(df.iloc[:, 4:66], inplace = True, axis = 1) 
df.rename(columns = {0:'TEST'}, inplace = True) 
df.drop(['TEST'], axis = 1,inplace=True) 
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

df=df.iloc[:287,:] # removing extra rows at the end

df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [196]:
df=df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [197]:
for i in range(len(df)):
    if df.iloc[i,2] == 'Not assigned':
       df.iloc[i,2] = df.iloc[i,1] 
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [198]:
df_Toronto=df
df_Toronto=df_Toronto.groupby('Postcode')['Borough','Neighbourhood'].agg(lambda x: ', '.join(set(x))).reset_index()

df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [199]:
df_Toronto.shape

(103, 3)

## Fetching geocodes for the Neighborhoods in above dataframe

In [187]:
CLIENT_ID = 'D1IXAYF5EGVSEDQ2PFFO01KVVXCYAICFE0F5YXMDPX1Z4STE' # your Foursquare ID
CLIENT_SECRET = 'WMFCF5KXW4N5PH0Q0J4NEZHE134H441NKFZFEGD4I1YJ2Z2H' # your Foursquare Secret
VERSION = '20200304'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D1IXAYF5EGVSEDQ2PFFO01KVVXCYAICFE0F5YXMDPX1Z4STE
CLIENT_SECRET:WMFCF5KXW4N5PH0Q0J4NEZHE134H441NKFZFEGD4I1YJ2Z2H


In [200]:
df_Toronto['Latitude']=''
df_Toronto['Longitude']=''
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


In [207]:
for i in range(len(df_Toronto)):
    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode('{}, Toronto, Ontario'.format(df_Toronto.iloc[i,0]))
    print(location)
    df_Toronto.iloc[i,3] = location.latitude
    df_Toronto.iloc[i,4] = location.longitude
print(i)
df_Toronto.head()

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
None


AttributeError: 'NoneType' object has no attribute 'latitude'

In [211]:
geolocator = Nominatim(user_agent="myGeocoder")
location = geolocator.geocode('M1E, Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [208]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.654,-79.3872
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.654,-79.3872
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


## Since fetching of geocode is failing for some of the postcodes using foursquare_agent. I will be using the csv to get the latitude and longitude values

In [212]:
df_ll=pd.read_csv('https://cocl.us/Geospatial_data')
df_ll.rename(columns = {'Postal Code':'Postcode'}, inplace = True) 
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [214]:
df_Toronto.drop(['Latitude','Longitude'], axis = 1,inplace=True) 
df_T=pd.merge(df_Toronto, df_ll, how='inner', on='Postcode', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
df_T.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [215]:
df_T.shape

(103, 5)